In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Product Capstone/Development/ML/Data.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [18]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [36]:
import random
import os
from shutil import copyfile

def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    files = os.listdir(SOURCE)
    files = [file for file in files if os.path.getsize(os.path.join(SOURCE, file)) > 0]

    # Calculate lengths for each set
    num_files = len(files)
    num_training = int(num_files * SPLIT_SIZE[0])
    num_testing = int(num_files * SPLIT_SIZE[1])

    # Shuffle the files
    random.shuffle(files)

    # Split into sets
    training_set = files[:num_training]
    validation_set = files[num_training + num_testing:]

    # Copy files to respective directories
    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        copyfile(this_file, destination)

    for filename in validation_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(VALIDATION, filename)
        copyfile(this_file, destination)

# Define source and destination directories
SOURCE_DIR = "/tmp/Dataset/"
TRAINING_DIR = "/tmp/training/"
VALIDATION_DIR = "/tmp/validation/"

# Define split size as a tuple (training_size, testing_size, validation_size)
split_size = (0.8, 0.2)

# Iterate over each label directory
for label in os.listdir(SOURCE_DIR):
    label_source_dir = os.path.join(SOURCE_DIR, label)
    label_training_dir = os.path.join(TRAINING_DIR, label)
    label_validation_dir = os.path.join(VALIDATION_DIR, label)

    # Create training, testing, and validation directories if they don't exist
    os.makedirs(label_training_dir, exist_ok=True)
    os.makedirs(label_validation_dir, exist_ok=True)

    # Split data for each label
    split_data(label_source_dir, label_training_dir, label_validation_dir, split_size)

In [21]:
def count_images_in_subdirs(directory):
    count = 0
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        count += len(os.listdir(label_dir))
    return count

# Count images in training, testing, and validation directories
num_training_images = count_images_in_subdirs(TRAINING_DIR)
num_validation_images = count_images_in_subdirs(VALIDATION_DIR)

# Print the counts
print("Number of training images:", num_training_images)
print("Number of validation images:", num_validation_images)

Number of training images: 84596
Number of validation images: 16834


In [28]:
# Define directories
train_dir = '/tmp/Image/Training/'
validation_dir = '/tmp/Image/Validation/'
testing_dir = '/tmp/Image/Testing/'

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255,
                                        horizontal_flip=True)

# Load and preprocess the data
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    seed=42,
    color_mode='rgb',
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False,
    color_mode='rgb',
)

Found 84596 images belonging to 36 classes.
Found 16834 images belonging to 36 classes.


In [29]:
labels = list(train_generator.class_indices.keys())
print(labels)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [30]:
pretrainedModel = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
     include_top=False,
     weights='imagenet'
)
pretrainedModel.trainable = False

inputs = pretrainedModel.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

optimizer_adam = tf.keras.optimizers.Adam(learning_rate = 0.005)

9406464/9406464 [==============================] - 0s 0us/step


In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [33]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.92):
            print("\n Accuracy is more than 95%, stopping...")
            self.model.stop_training = True

callback = myCallback()

In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [35]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[callback]
 )

Epoch 1/100
1322/1322 [==============================] - 973s 731ms/step - loss: 0.8711 - accuracy: 0.7276 - val_loss: 0.2920 - val_accuracy: 0.9051
Epoch 2/100
1322/1322 [==============================] - 953s 721ms/step - loss: 0.4444 - accuracy: 0.8573 - val_loss: 0.1935 - val_accuracy: 0.9385
Epoch 3/100
1322/1322 [==============================] - 953s 721ms/step - loss: 0.3678 - accuracy: 0.8812 - val_loss: 0.1826 - val_accuracy: 0.9399
Epoch 4/100
 249/1322 [====>.........................] - ETA: 12:31 - loss: 0.3526 - accuracy: 0.8885

KeyboardInterrupt: 